# Exercise 1: Optimize lepton selection

* First, print the distributions of the relevant variables for *all* the Monte Carlo samples (i.e. all the *channels* of the $Z$-boson decay to be studied). Which variables are these? Give sensible ranges to include all the events in the samples (both MC and OPAL data) 
* Do the same for **one** of the OPAL data samples (your lab assistant will decide which one you choose).
* Describe the results.
* Optimize the object selection by applying cuts. Make a strategy on how to proceed to find the optimal selection. which information do you need?
* Determine the efficiency and the amount of background for each $Z$ decay channel. Use the simulated events $e^+e^-$, $\mu^+\mu^-$, $\tau^+\tau^-$ and hadrons ($qq$). Represent the result in a matrix form and think carefully about how you have to correct the measured rates. Don't forget to calculate the errors!
* How do we estimate the statistical fluctuations per bin?

**Import libraries**:

In [0]:
import uproot
import awkward as ak
import mplhep
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import scipy.constants as sc

**Open Monte Carlo samples**

We will open data and Monte Carlo samples using **uproot**. Uproot is a reader and a writer of the ROOT file format using only Python and Numpy. Unlike PyROOT and root_numpy, uproot does not depend on C++ ROOT so that no local compilation of the ROOT libraries is needed to access the data.

You can find more info on uproot following the references:
* Github repo: https://github.com/scikit-hep/uproot4
* Tutorial: https://masonproffitt.github.io/uproot-tutorial/
* Video tutorial on uproot and awkward arrays:  https://www.youtube.com/embed/ea-zYLQBS4U 

First, let's specify the folder path for both data and Monte Carlo (MC) samples

In [0]:
# Read the Monte Carlo data
path_mc = 'opal_data/mc/'

# Open the files
file_mc_ee = uproot.open(path_mc+'ee.root')
file_mc_mm = uproot.open(path_mc+'mm.root')
file_mc_tt = uproot.open(path_mc+'tt.root')
file_mc_qq = uproot.open(path_mc+'qq.root')


# Name the ttree name
ttree_name = 'myTTree'

# Print list of 'branches' of the TTree (i.e. list of variable names)
variable_names = file_mc_ee[ttree_name].keys()

# Load branches
branches_ee = file_mc_ee[ttree_name].arrays()
branches_mm = file_mc_mm[ttree_name].arrays()
branches_tt = file_mc_tt[ttree_name].arrays()
branches_qq = file_mc_qq[ttree_name].arrays()
branches_tot = ak.concatenate((branches_ee,
                               branches_mm,
                               branches_tt,
                               branches_qq))


# convert data into dictionaries of numpy arrays
array_ee = {}
array_mm = {}
array_tt = {}
array_qq = {}
array_tot = {}

for variable in variable_names:
    array_ee[variable] = ak.to_numpy(branches_ee[variable])
    array_mm[variable] = ak.to_numpy(branches_mm[variable])
    array_tt[variable] = ak.to_numpy(branches_tt[variable])
    array_qq[variable] = ak.to_numpy(branches_qq[variable])
    array_tot[variable] = ak.to_numpy(branches_tot[variable])

all_channels = ['tot', 'ee', 'mm', 'tt', 'qq']
dist_channels = ['ee', 'mm', 'tt', 'qq']
array = {'ee' : array_ee,
         'mm' : array_mm,
         'tt' : array_tt,
         'qq' : array_qq,
         'tot' : array_tot}


# print('These are the different variables: ', variable_names)
# print('E_lep = {0:4.2f} GeV = const.'.format(branches_ee['E_lep'][0]))

| Variable name | Description |
| --- | --- | 
| run| Run number |
| event | Event number |
| Ncharged | Number of charged tracks |
| Pcharged | Total scalar sum of track momenta |
| E_ecal| Total energy measured in the electromagnetic calorimeter |
| E_hcal | Total energy measured in the hadronic calorimeter |
| E_lep | LEP beam energy (=$\sqrt{s}/2$) |
| cos_thru | cosine of the polar angle between beam axis and thrust axis |
| cos_thet | cosine of the polar angle between incoming positron and outgoing positive particle |


For our statistical analysis the run and event number are not interesting. The beam energy $E_\mathrm{lep}=45.64\,\mathrm{GeV}$ is constant for the data set. The scattering angles will not be used for identifying the channels and are subsect of analysis further below. Therefore we first concentrate only on the subset of variables Ncharged, Pcharged, E_ecal and E_hcal.

In [0]:
### We define set of variables of interest
### and corresponding dictionaries of plot settings, etc.
variables = ['Ncharged', 'Pcharged', 'E_ecal', 'E_hcal']

# units corresponding to the variables
unit = {'Ncharged' : '',
         'Pcharged' : ' [GeV]',
         'E_ecal' : ' [GeV]',
         'E_hcal' : ' [GeV]'}

# bins for histograms
bins_mc = {'Ncharged' : np.linspace(0, 40, 41),
           'Pcharged' : np.linspace(0, 120, 201),
           'E_ecal' : np.linspace(0, 120, 201),
           'E_hcal' : np.linspace(0, 40, 201)}

# limits for the plots
ylims_mc = {'Ncharged' : (0, 1e4),
            'Pcharged' : (0, 1e4),
            'E_ecal' : (0, 1e4),
            'E_hcal' : (0, 1e4)}


### Dictionaries concerning different channels
plot_label = {'ee' : r'$e$',
              'mm' : r'$\mu$',
              'tt' : r'$\tau$',
              'qq' : r'$q$',
              'tot' : 'sum'}

color = {'ee' : 'tab:blue',
         'mm' : 'tab:orange',
         'tt' : 'tab:green',
         'qq' : 'tab:red',
         'tot' : 'grey'}


### plotting
plt.style.use(mplhep.style.ATLAS)  # plot style of ATLAS
fig, axes = plt.subplots(1, 4, figsize=(28, 5))

# plot histograms for all variables and channels
for (i, variable) in zip(range(4), variables):
    for channel in all_channels:
        axes[i].hist(array[channel][variable],
                     bins=bins_mc[variable],
                     histtype='step',
                     linewidth=2,
                     color=color[channel],
                     label=plot_label[channel])


    # plot settings
    axes[i].set_ylim(ylims_mc[variable])
    axes[i].set_title(variable)
    axes[i].set_xlabel(variable + unit[variable])
    axes[i].set_ylabel(r'# events $N$')
    axes[i].legend()


plt.tight_layout()
plt.show()

In the histograms above we see different kind of distributions for different variables and decay channels. Some distributions closely resemble a Gaussian such that a cut can easily be defined via mean and standard deviation. If for example we choose an interval of 3$\sigma$ we keep approximately 99% of the events. Since the distributions partially overlap, this also causes many false positive events. We hope to make the filters unique by applying such cuts on multiple variables.

Other distributions are not easily approximated by a Guassian. These are e.g. the Ncharged curves for $e$, $\mu$ and $\tau$, since they have a small mean. Also Pcharged is problematic since often the momentum is missing in the data (is this interpretation correct?), e.g., for $e$ about 50% of the events have Pcharged=0, while the rest of the distribution is centred at higher values.


All these effects have to be considered to define meaningful cuts. Nevertheless, let us start the analysis by calculating the mean and standard deviation of all distributions.

In [0]:
# data structure for mean values
mean = {'ee' : {},
        'mm' : {},
        'tt' : {},
        'qq' : {}}

# data structure for standard deviation
std = {'ee' : {},
       'mm' : {},
       'tt' : {},
       'qq' : {}}

# auxillary data structure for masks and masked arrays
aux_mask = {'ee' : {},
            'mm' : {},
            'tt' : {},
            'qq' : {}}

aux_array = {'ee' : {},
             'mm' : {},
             'tt' : {},
             'qq' : {}}


# calculate values for all variables and channels
for var in variables:
    for ch in dist_channels:
        # aux array is copied from regular array
        aux_array[ch][var] = array[ch][var]

        # Pcharged exhibits unexpected behavior:
        # a significant number of values is 0.
        # there are few outliers with very high values distorting the statistics
        # therefore consider only 'regular' events to describe the distributions
        if var == 'Pcharged':
            # apply lower and upper cut
            aux_mask[ch][var] = (array[ch][var] > 0.)
            aux_mask[ch][var] *= (array[ch][var] < 120.)
            aux_array[ch][var] = aux_array[ch][var][aux_mask[ch][var]]

        # calculate statistics of possibly masked arrays
        mean[ch][var] = aux_array[ch][var].mean()
        std[ch][var] = aux_array[ch][var].std()


    # print as a table
    print(var)
    print('    mean  +-   std, [min,      max]')
    for ch in dist_channels:
        print('{0}: {1:5.2f} +- {2:5.2f}, [{3:5.2f}, {4:6.2f}]'.format(
                ch, mean[ch][var], std[ch][var],
                aux_array[ch][var].min(), aux_array[ch][var].max()))
    print('\n')

**Define cuts:**

Apply cuts to the different distributions. Individual notes to the different cut selections are commented.
The general reference where to apply a cut is: mean +- 3 sigma, to include most of all events in our selection.
In cases of specific asymmetric distributions or something similar, a correction by eye is performed.

In [0]:
### define cuts
cuts = {'ee' : {},
        'mm' : {},
        'tt' : {},
        'qq' : {}}

cuts['ee'] = {'Ncharged' : (0, 6),
              'Pcharged' : (mean['ee']['Pcharged'] - 3*std['ee']['Pcharged'], 100),
              'E_ecal' : (mean['ee']['E_ecal'] - 3*std['ee']['E_ecal'],
                          mean['ee']['E_ecal'] + 3*std['ee']['E_ecal']),
              'E_hcal' : (0, mean['ee']['E_hcal'] + 3*std['ee']['E_hcal'])}

cuts['mm'] = {'Ncharged' : (0, 4),
              'Pcharged' : (mean['mm']['Pcharged'] - 3*std['mm']['Pcharged'],
                            mean['mm']['Pcharged'] + 3*std['mm']['Pcharged']),
              'E_ecal' : (0, mean['mm']['E_ecal'] + 3*std['mm']['E_ecal']),
              'E_hcal' : (0, mean['mm']['E_hcal'] + 3*std['mm']['E_hcal'])}

cuts['tt'] = {'Ncharged' : (0, 7),
              'Pcharged' : (mean['tt']['Pcharged'] - 2*std['tt']['Pcharged'],
                            mean['tt']['Pcharged'] + 3*std['tt']['Pcharged'] - 15),
              'E_ecal' : (0, mean['tt']['E_ecal'] + 3*std['tt']['E_ecal'] - 5),
              'E_hcal' : (0, mean['tt']['E_hcal'] + 3*std['tt']['E_hcal'])}

cuts['qq'] = {'Ncharged' : (7, 39),
              'Pcharged' : (mean['qq']['Pcharged'] - 2*std['qq']['Pcharged'] - 10,
                            mean['qq']['Pcharged'] + 2*std['qq']['Pcharged'] - 4),
              'E_ecal' : (mean['qq']['E_ecal'] - 3*std['qq']['E_ecal'],
                          mean['qq']['E_ecal'] + 3*std['qq']['E_ecal'] - 10),
              'E_hcal' : (0, mean['qq']['E_hcal'] + 3*std['qq']['E_hcal'])}


# print as a table
for var in variables:
    print(var)
    print('    left cut,  right cut')
    for ch in dist_channels:
        print('{0}: {1:7.2f} , {2:10.2f}'.format(
                ch,
                cuts[ch][var][0],
                cuts[ch][var][1]))
    print('\n')

**Tool to adjust Cuts:**

Here we can specify which data and which cuts should be shown. This allows to visualize nicely where cuts should be changed in order to minimize e.g. a specific off diagonal element of the efficiency matrix.

In [0]:
### Plotting only specific channels and cuts for all variables
spec_channels = ['tot', 'ee', 'mm', 'tt', 'qq']
spec_cuts = ['ee']

print(f'data from: {spec_channels}')
print(f'cuts from: {spec_cuts}')


### plotting
plt.style.use(mplhep.style.ATLAS)  # plot style of ATLAS
fig, axes = plt.subplots(1, 4, figsize=(28, 5))

# plot histograms for all variables and channels
for (i, variable) in zip(range(4), variables):
    for channel in spec_channels:
        axes[i].hist(array[channel][variable],
                     bins=bins_mc[variable],
                     histtype='step',
                     linewidth=2,
                     alpha= 0.9,
                     color=color[channel],
                     label=plot_label[channel])

    for cut in spec_cuts:
        axes[i].axvline(cuts[cut][variable][0], ls='--', lw=3, color=color[cut])
        axes[i].axvline(cuts[cut][variable][1], ls='--', lw=3, color=color[cut],
                        label=f'cut: {plot_label[cut]}')


    # plot settings
    axes[i].set_ylim(ylims_mc[variable])
    axes[i].set_title(variable)
    axes[i].set_xlabel(variable + unit[variable])
    axes[i].set_ylabel(r'# events $N$')
    axes[i].legend()


plt.tight_layout()
plt.show()

In the plots, specific cuts in combination with individual data can be observed.
- The parameter "spec_channels" specifies the data sets that will be plotted
- The parameter 'spec_cuts' specifies the specific particle types (ee, mm, tt or qq) for which the selection cuts will be shown

This piece of code can be used to play a bit around with the data sets and the cuts.

It can be observed, that:
- Ncharged is a good indicator to seperate the hadronic from the leptonic channels

- Pcharged is generally a good indicator to distinguish between mm and tt decays.
- The problem here is, that in many cases the momenta were not tracked correctly, resulting in a tracked value of Pcharged=0.
- The mm- respectively tt- Pcharged=0 events can not be distinguished.
- As there is no other effective criterion to prevent mm-events to be classified as tt-events, the selection rate here is the worst in our whole data.
- It is possible to prevent tt-events to be classified as mm-events via Ncharged. Unfortunately this is not possible the other way round.

- Eecal allows us to seperate ee and mm events very effectively

- Hcal can be used to avoid mistakenly assigning electrons to other events.

In [0]:
def identify(array_xx, channel):
    '''Returns a mask designed to identify events of channel in array_xx'''
    mask = True
    for variable in variables:
        # lower and upper cut
        mask *= array_xx[variable] >= cuts[channel][variable][0]
        mask *= array_xx[variable] <= cuts[channel][variable][1]
        # to include data with artifacts in 'Pcharged'
        # this does not ocurr for hadrons
        if variable=='Pcharged' and channel in ['ee', 'mm', 'tt']:
            mask += array_xx['Pcharged'] == 0
    return mask

In [0]:
# calculate efficiency matrix
filter_eff = np.zeros((4, 4))
filter_error = np.zeros((4, 4))
n_mc = np.zeros((4, 4))
N_mc = np.zeros(4)

for (j, col_ch) in zip(range(4), dist_channels):
    N_mc[j] = 100000  # len(array[col_ch]['Pcharged'])  # 100000?
    for (i, row_ch) in zip(range(4), dist_channels):
        n_mc[i, j] = sum(identify(array[col_ch], row_ch))


filter_eff = n_mc / N_mc[None,:]
filter_error = np.sqrt((n_mc+1)*(n_mc+2)/(N_mc[None,:]+2)/(N_mc[None,:]+3) - (n_mc+1)**2/(N_mc[None,:]+2)**2)

In [0]:
print("efficiency matrix: \n", np.array_repr(filter_eff, precision=7, suppress_small=True), '\n \n',
      "errors of efficiency matrix: \n", np.array_repr(filter_error, precision=7, suppress_small=True))

### Matrix Inversion
To determine the uncertainties of the matrix elements after the inversion we diffierent methods and compare:
- analytical formula (see reference below)
- generate random efficiency matrices, invert all and determine the statistics via:
  * direct formula of mean and std
  * Gaussian fit to each matrix element

**References**:
* Propagation of Errors for Matrix Inversion: https://arxiv.org/abs/hep-ex/9909031v1

In [0]:
### invert matrix and propagate errors
inv_filter_eff = np.linalg.inv(filter_eff)
inv_filter_error = np.sqrt(np.linalg.multi_dot([inv_filter_eff**2,
                                                filter_error**2,
                                                inv_filter_eff**2]))

# covariance matrix of the matrix elements
inv_filter_cov = np.zeros((4, 4, 4, 4))

for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                left_vec = (inv_filter_eff[i, :]*inv_filter_eff[k, :])
                right_vec = (inv_filter_eff[:, j]*inv_filter_eff[:, l])
                inv_filter_cov[i, j, k, l] = left_vec @ filter_error[:, :]**2 @ right_vec


print(inv_filter_cov[0, :, 3, :])

In [0]:
### Number of toy experiments to be done
ntoy = 1000

### Create numpy matrix of list to append elements of inverted toy matrices
inverse_toys = np.empty((4,4))

# Create toy efficiency matrix out of gaussian-distributed random values
for i in range(0,ntoy,1):
    toy_matrix = np.zeros((4,4))
    toy_matrix = np.random.normal(filter_eff,
                                  filter_error,
                                  size=(4,4))
    
    # Invert toy matrix
    inverse_toy = np.linalg.inv(toy_matrix)
    
    # Append values
    inverse_toys = np.dstack((inverse_toys,inverse_toy))

### calculate the statistics directly
inv_filter_eff_stat = np.mean(inverse_toys, axis=2)
inv_filter_error_stat = np.std(inverse_toys, axis=2)

In [0]:
### Get statistics of toy experiments from Gaussian fit

from scipy.optimize import curve_fit


inv_filter_eff_fit = np.zeros((4,4))
inv_filter_error_fit = np.zeros((4,4))

# Define gaussian function to fit to the toy distributions:
def gauss(x, A, mu, sigma):
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))


fig = plt.figure(figsize=(20, 10), dpi=80)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.2)
ax00 = plt.subplot(4,4,1)
ax01 = plt.subplot(4,4,2)
ax02 = plt.subplot(4,4,3)
ax03 = plt.subplot(4,4,4)

ax10 = plt.subplot(4,4,5)
ax11 = plt.subplot(4,4,6)
ax12 = plt.subplot(4,4,7)
ax13 = plt.subplot(4,4,8)

ax20 = plt.subplot(4,4,9)
ax21 = plt.subplot(4,4,10)
ax22 = plt.subplot(4,4,11)
ax23 = plt.subplot(4,4,12)

ax30 = plt.subplot(4,4,13)
ax31 = plt.subplot(4,4,14)
ax32 = plt.subplot(4,4,15)
ax33 = plt.subplot(4,4,16)

axes = [[ax00,ax01,ax02,ax03],
        [ax10,ax11,ax12,ax13],
        [ax20,ax21,ax22,ax23],
        [ax30,ax31,ax32,ax33]]

## Find suitable ranges to fit/plot gaussian distributions
ranges = np.zeros((4, 4, 2))
ranges[:, :, 0] =  inv_filter_eff - 4*inv_filter_error
ranges[:, :, 1] =  inv_filter_eff + 4*inv_filter_error

## Find suitable initial parameters for gaussian distributions
p0s = np.zeros((4, 4, 3))
p0s[:, :, 0] = 10000
p0s[:, :, 1] = inv_filter_eff
p0s[:, :, 2] = inv_filter_error


# Fill histograms for each inverted matrix coefficient:
for j in range(0, 4):
    for k in range(0, 4):

        # Diagonal and off-diagonal terms have different histogram ranges
        hbins, hedges, _ = axes[j][k].hist(inverse_toys[j,k,:],
                                           bins=30,
                                           range=ranges[j, k, :],
                                           histtype='step',
                                           linewidth=2,
                                           label=f'toyhist{j}{k}')
        axes[j][k].legend()


        # Get the fitted curve
        h_mid = 0.5*(hedges[1:] + hedges[:-1]) #Calculate midpoints for the fit
        coeffs, _ = curve_fit(gauss, h_mid, hbins, p0=p0s[j, k], maxfev=10000)
        h_fit = gauss(h_mid, *coeffs)

        inv_filter_eff_fit[j, k] = coeffs[1]
        inv_filter_error_fit[j, k] = coeffs[2]

        axes[j][k].plot(h_mid, h_fit,label=f'Fit{j}{k}')

plt.tight_layout()

In [0]:
### comparison
print("inverse matrix: \n", np.array_repr(inv_filter_eff, precision=7, suppress_small=True), '\n')
print("analytical errors: \n", np.array_repr(inv_filter_error, precision=7, suppress_small=True), '\n')
print("direct statistical errors: \n", np.array_repr(inv_filter_error_stat, precision=7, suppress_small=True), '\n')
print("fit errors: \n", np.array_repr(inv_filter_error_fit, precision=7, suppress_small=True))

The analytical expression is exact as discussed in the reference. We see that the fit method closely matches these results. The other method however does not. Why is that so? We do not know. BUT: Since the analytical expression is exact, we do not have to find the reason. We stick with the exact propagation formula.

# Exercise 2: Separate $t$- and $s$-channel contributions

Only Feynman diagrams contributing to the production of $Z$ boson are to be considered for the measurements. The **electron** Monte Carlo sample incorporate contributions from $t$- and $s$-channels.
* Select/correct contributions producing $Z$ boson decays. (Hint: Which role does the $\cos(\theta)$ distribution play in separating $t$- and $s$-channels?)

In [0]:
# As the t-channel only exists for ee-events, only these are considered in ex.2

### We define set of angle variables
# add definitions to dictionaries from above
angle_variables = ['cos_thru', 'cos_thet']

# units corresponding to these variables
unit['cos_thru'] = ''
unit['cos_thet'] = ''

# bins for histograms
bins_mc['cos_thru'] = np.linspace(-1, 1, 201)
bins_mc['cos_thet'] = np.linspace(-1, 1, 201)

# limits for the plots
ylims_mc['cos_thru'] = (0, 2e3)
ylims_mc['cos_thet'] = (0, 2e3)

# Function that describes the s-channel distribution
def s_distribution(cos_thet, A):
    return A*(1+cos_thet**2)

# function that describes the t-channel
def t_distribution(cos_thet, A):
    return A*(1-cos_thet)**-2

### plotting
plt.style.use(mplhep.style.ATLAS)  # plot style of ATLAS
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# plot histograms for the cos_thet and cos_thru distribution of the ee-events
for (i, variable) in zip(range(2), angle_variables):
    for channel in ['ee']:
        axes[i].hist(array[channel][variable],
                     bins=bins_mc[variable],
                     histtype='step',
                     linewidth=2,
                     color=color[channel],
                     label=plot_label[channel])

    # plot settings
    axes[i].set_ylim(ylims_mc[variable])
    axes[i].set_title(variable)
    axes[i].set_xlabel(variable + unit[variable])
    axes[i].set_ylabel(r'# events $N$')
    axes[i].legend()

plt.tight_layout()
plt.show()


# Cite Paper: Precise Determination of the Z Resonance Parameters at LEP: "Zedometry"
# p.23 (ch. 7.2.1 t-channel contribution to ee -> ee)
# Some key points:
# - t-channel amplitude is non-resonant, dominated by photon exchange
# - relative size of t-channel amplitude depends on the scattering angle (theta)
# - cut is made at |cos(theta)| < 0.7
# - resulting from high-statistics Monte Carlo samples
# OPAL_Paper_3 results in the same

# Apply the cut at |cos(theta)| < 0.7 to seperate s- and t-channels for ee-events
cuts['ee']['cos_thet'] = (-0.7, 0.7)

The relative size of the t-channel amplitude depends on cos_theta and becomes the major component at high values of this variable.
To make sure the polar angle lies well within the barrel region of the detector, a cut is made at |cos(theta)| < 0.7 (as explained in the paper "Zedometry").
In order to enhance the s-channel component of the selected data sample, a linear combination of the s-and t-channel distributions is fitted to the data. Then, only the s-channel part is plotted, as well.
This s-channel curve (i.e. the area below this curve in the range[-1,1]) indicates now, how many events correspond to the s-channel. This number of events can be determined by integration (Take the bin width into account!).
The s+t-channel curve (i.e. the area below in the range (-0.7,0.7)) indicates how many events we find in our data after the cut. In order to be able to determine the number of real s-channel events in the range [-1,1] using the number of measured counts in the range (-0.7,0.7), we will divide the areas under the curves in the described ranges and thus obtain a correction factor with which we can determine the s-channel-data from our measurement data.

In [0]:
# fit a linear combination of s- and t-channel distributions
def s_t_distribution(cos_thet, A, B):
    '''Linear combination of s- and t-channel distributions'''
    return A*(1+cos_thet**2) + B*(1-cos_thet)**-2


# function to integrate the s+t channel function in the range (-0.7,0.7)
# the coefficients were determined by wolframAlpha
def s_t(A, B):
    return A * 1.62867 + B * 2.7451


# define variable and channel
variable = 'cos_thet'
channel = 'ee'

###plotting
fig, ax = plt.subplots()
# histogram the data
hbins, hedges, _ = ax.hist(array[channel][variable], bins=np.linspace(-1,1,201),
                        histtype='step', linewidth=2, color='C0',
                        label=plot_label[channel])
h_mid = 0.5*(hedges[1:] + hedges[:-1])  # Calculate midpoints for the fit
ax.errorbar(h_mid[10:-10:10], hbins[10:-10:10], yerr=np.sqrt(hbins[10:-10:10]),
            c='black', fmt='none', zorder=5, capsize=2, label='stat. uncertainty')

# perform the fit
coeffs, cov = curve_fit(s_t_distribution, h_mid[10:-10], hbins[10:-10],
                        sigma=np.sqrt(hbins[10:-10]), absolute_sigma=True,
                        maxfev=10000)  # fit in between -0.895 and 0.895

# plot the s+t channel fit
ax.plot(bins_mc[variable], s_t_distribution(bins_mc[variable], coeffs[0], coeffs[1]),
        color="C2", label='(s+t) - fit')

# plot the s-channel fit
ax.plot(bins_mc[variable], s_distribution(bins_mc[variable], coeffs[0]),
        color="C3", label='s - fit')


# Show the cuts in the graph
ax.axvline(cuts[channel][variable][1], ls='--', color = 'C1', label=r'data cuts')
ax.axvline(cuts[channel][variable][0], ls='--', color = 'C1')
ax.axvline(h_mid[10], ls='--', color = 'black', label=r'fit limits')
ax.axvline(h_mid[-10], ls='--', color = 'black')

#print('fit edges:', h_mid[10], h_mid[-11])

# Plot settings
ax.set_ylim(0,1e3)
ax.set_title(r'cos_theta-distribution')
ax.set_xlabel(r'cos($\theta$)')
ax.set_ylabel(r'# events $N$')
ax.legend()
plt.show()

# calculate the bin width
bin_width = hedges[1]-hedges[0]

# How many events do we count in the range (-0.7,0.7)?
# Integration
A_count = s_t(coeffs[0], coeffs[1]) / bin_width
# Error propagation
u_A_count = np.sqrt((s_t(cov[0,0], 0))**2
                      + s_t(0, cov[1,1])**2
                      + 2 * s_t(cov[0,0],0)
                      * s_t(0, cov[1,1])*cov[0,1]) / bin_width

# How many events do really correspond to the s-channel?
# Integration
A_real = 8/3 * coeffs[0] / bin_width
# Error propagation
u_A_real = 8/3 * cov[0,0] / bin_width

# Factor #real_events / #counted_events
ratio_s_t = A_real / A_count
# Error propagation
u_ratio_s_t = np.sqrt((u_A_real/A_count)**2 + (A_real/A_count**2 * u_A_count)**2)

# Print the results
print('A_count =', A_count.round(0), '+-', u_A_count.round(0), ', relative error:', (u_A_count/A_count).round(3))
print('A_real =', A_real.round(0), '+-', u_A_real.round(0), ', relative error:', (u_A_real/A_real).round(3))
print('Ratio real s-channel data / counted data =', ratio_s_t.round(3), '+-', u_ratio_s_t.round(3), ', relative error:', (u_ratio_s_t/ratio_s_t).round(3))

In [0]:
# Function to apply the cos_thet cuts to the ee-data
def identify_s(array_ee):
    '''Returns a mask designed to identify s-channel in array_ee'''
    mask = True
    mask *= array_ee['cos_thet'] > cuts['ee']['cos_thet'][0]
    mask *= array_ee['cos_thet'] < cuts['ee']['cos_thet'][1]
    return mask

In [0]:
# Plot the Pcharged - distribution for the ee-data with and without a cut in cos_thet

# Setup figure
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

variable = 'Pcharged'

# create mask without cos_thet cut
mask_ee = identify(array['tot'],'ee')
# Add the cos_thet cut to a second mask
mask_thet_ee = mask_ee * identify_s(array['tot'])

# Plot Pcharged without cos_thet cut
axes[0].hist(array['tot'][variable][mask_ee],
             bins=np.linspace(0,120,201), histtype='step',
             linewidth=2, label=r'ee-data without cos$\theta$ - cut')

# Plot Pcharged with cos_thet cut
axes[1].hist(array['tot'][variable][mask_thet_ee],
             bins=np.linspace(0,120,201), histtype='step',
             linewidth=2, label=r'ee-data with cos$\theta$ - cut')


# Plot settings
axes[0].set_title(r"Pcharged without cos$\theta$ - cut")
axes[1].set_title(r"Pcharged with cos$\theta$ - cut")
axes[0].set_xlabel(variable + unit[variable])
axes[1].set_xlabel(variable + unit[variable])
axes[0].set_ylabel(r'# events $N$')
axes[1].set_ylabel(r'# events $N$')
axes[0].legend()
axes[1].legend()

plt.tight_layout()
plt.show()

It can be observed that by applying the cos_theta cut we loose most of the Pcharged=0 events. This is due to the fact, that at small scattering angles the electron flies nearly parallel to the magnetic field, so the momentum cannot be measured.

# Exercise 3: Measurement of the total production cross sections

For **each** of the seven centre-of-mass energies:
* Determine the number of events in the handronic channel *and* in the three leptonic channels
* Substract the background and correct for selection efficiencies accordingly
* Then, calculate the differnetial cross sections for the hadronic *and* the leptnic channels
* Add the radiation corrections from The table given below. **Don't forget to take the uncertainties (errors) into account!**

| $\sqrt{s}$   \[GeV\]| Correction hadronic channel    \[nb\] |  Correction leptonic channel   \[nb\]|
| --- | --- | --- |
| 88.47 | +2.0  | +0.09 |
| 89.46 | +4.3  | +0.20 |
| 90.22 | +7.7  | +0.36 |
| 91.22 | +10.8 | +0.52 |
| 91.97 | +4.7  | +0.22 |
| 92.96 | -0.2  | -0.01 |
| 93.76 | -1.6  | -0.08 |

Feel free to access these values using the dictionary 'xs_corrections' given below.
* Once the total cross section for all four decay channels at all seven energies have been measured, fit a **Breit-Wigner distribution** to measure the $Z$ boson mass ($m_Z$) and the resonance width ($\Gamma_Z$) and the peak cross section s of the resonance for the hadronic and the leptonic channels. Again, **propagate the uncertainties carefully**.
* Compare your results to the OPAL cross section s and the theoretical predictions. How many degrees of freedom does the fit have? How can you udge if the model is compatible with the measured data? Calculate the  **confidence levels**.
* Calculate the partial widths for all channels from the measured cross sections on the peak. Which is the best partial width to start with? Compare them with the theoretical predictions and the values that you have calculated in the beginning.
* Determine from your results the **number of generations of light neutrinos**. Which assumptions are necessary?
* Discuss in detail the systematic uncertainties in the whole procedure of the analysis. Which assumptions were necessary?

These are some **references** that might be interesting to look up:
* Particle Data Book: https://pdg.lbl.gov/2020/download/Prog.Theor.Exp.Phys.2020.083C01.pdf
** Resonances: https://pdg.lbl.gov/2017/reviews/rpp2017-rev-resonances.pdf
* Precision Electroweak Measurements on the Z Resonance (Combination LEP): https://arxiv.org/abs/hep-ex/0509008
* Measurement of the $Z^0$ mass and width with the OPAL detector at LEP: https://doi.org/10.1016/0370-2693(89)90705-3
* Measurement of the $Z^0$ line shape parameters and the electroweak couplings of charged leptons: https://inspirehep.net/literature/315269
* The OPAL Collaboration, *Precise Determination of the $Z$ Resonance Parameters at LEP: "Zedometry"*: https://arxiv.org/abs/hep-ex/0012018
* Fitting a Breit-Wigner curve using uproot: https://masonproffitt.github.io/uproot-tutorial/07-fitting/index.html

In [0]:
xs_corrections = { 'energy' : [88.47, 89.46, 90.22, 91.22, 91.97, 92.96, 93.76] ,
                   'hadronic' : [2.0, 4.3, 7.7, 10.8, 4.7, -0.2, -1.6],
                   'leptonic' : [0.09, 0.20, 0.36, 0.52, 0.22, -0.01, -0.08]}

In [0]:
### Data management

path_data = 'opal_data/data'

# Open the files
file_lep = uproot.open(path_data+'/daten_1.root')

# Name the ttree name
ttree_name = 'myTTree'

# Load branches
branches_lep = file_lep[ttree_name].arrays()

# convert data into dictionaries of numpy arrays
array_lep = {}

for variable in variable_names:
    array_lep[variable] = ak.to_numpy(branches_lep[variable])

N_data = len(array_lep['E_ecal'])

print('Number of events:', N_data)

print('These are the different variables: ',
      file_lep[ttree_name].keys(),
      ' the same as before')

print('E_lep = {0:4.2f} GeV = const.'.format(branches_ee['E_lep'][0]))

In [0]:
# identify the events of each energy

# dictionary for different beam energies
array_E_lep = {}
E_lep = np.zeros(7)

# roughly estimated bin edges
E_bin_edges = [44, 44.5, 44.9, 45.3, 45.8, 46.2, 46.7, 47.5]

for i in range(7):
    array_E_lep[i] = {}  # sub dictionary for different variables
    mask = True
    mask *= (array_lep['E_lep'] >= E_bin_edges[i])
    mask *= (array_lep['E_lep'] <= E_bin_edges[i+1])

    # mask arrays of all variables
    for variable in variable_names:
        array_E_lep[i][variable] = array_lep[variable][mask]

    E_lep[i] = np.mean(array_E_lep[i]['E_lep'])

In [0]:
# M_all_chs contains vectors of event numbers (of all channels)
M_all_chs = np.zeros((7, 4))

for i in range(7):
    for (j, ch) in zip(range(4), dist_channels):
        M_all_chs[i, j] = np.sum(identify(array_E_lep[i], ch))


print('E_lep | [ee,     mm,    tt,    qq]')
print('------------------------------------')
for i in range(7):
    print(round(E_lep[i], 2), '|', M_all_chs[i, :])

In [0]:
# M are vectors of event numbers (only s channel)
M = M_all_chs
u_M = np.sqrt(M_all_chs)

# for ee we have to separate s and t channel
# keep only s channel and correct for missing events
for i in range(7):
    mask_ee_s = identify(array_E_lep[i], 'ee')
    mask_ee_s *= identify_s(array_E_lep[i])
    M[i, 0] = np.sum(mask_ee_s)
    M[i, 0] *= ratio_s_t  # correct for s-channel selection
    u_M[i, 0] = M[i, 0] * np.sqrt((u_ratio_s_t/ratio_s_t)**2 + 1/M[i, 0])  # error propagation
    
    
print('E_lep | [ee,     mm,    tt,    qq][stat. uncertainty][rel. uncertainty]')
print('--------------------------------------------------------------------------------------')
for i in range(7):
    print(round(E_lep[i], 2),
          '|',
          np.round(M[i, :], 0),
          np.round(u_M[i, :], 0),
          np.round(u_M[i, :]/M[i, :], 3))

In [0]:
# N are vectors of event numbers corrected for efficiency
N = np.zeros((7, 4))
N_cov = np.zeros((7, 4, 4))
u_N = np.zeros((7, 4))

for i in range(7):
    N[i, :] = inv_filter_eff @ M[i, :]
    for j in range(4):
        for k in range(4):
            N_cov[i, j, k] = M[i, :] @ inv_filter_cov[j, :, k, :] @ M[i, :]
            N_cov[i, j, k] += (inv_filter_eff[j, :] * inv_filter_eff[k, :]) @ u_M[i, :]**2
    u_N[i, :] = np.sqrt(np.diagonal(N_cov[i, :, :]))


print('E_lep | [ee,     mm,    tt,    qq][stat. uncertainty][rel. uncertainty]')
print('--------------------------------------------------------------------------------------')
for i in range(7):
    print(round(E_lep[i], 2), '|', np.round(N[i, :], 0), np.round(u_N[i, :], 0), np.round(u_N[i, :]/N[i, :], 3))

In [0]:
#for i in range(7):
#    print(np.array_repr(N_cov[i, :, :], precision=4, suppress_small=True))

The off diagonal entries are small. further consider only variances, no covariances.

In [0]:
# import luminosity file
import pandas as pd

lumi_df = pd.read_csv('opal_data/lumi_files/daten_1.csv')

print('these are the different columns: ', lumi_df.keys())

In [0]:
# calculate cross sections

cross_array = N / np.array(lumi_df['lumi'])[:, None]
u_cross_array_stat = cross_array * np.sqrt((u_N/N)**2 + np.array(lumi_df['stat']/lumi_df['lumi'])[:, None]**2)
u_cross_array_all = cross_array * np.sqrt((u_N/N)**2 + np.array(lumi_df['all']/lumi_df['lumi'])[:, None]**2)

cross_array[:, :3] += np.array(xs_corrections['leptonic'])[:, None]
cross_array[:, 3] += np.array(xs_corrections['hadronic'])

# for consistency go back to dictionaries
# no more matrix calculations needed
cross_sec = {}
u_cross_stat = {}
u_cross_all = {}

for i, ch in zip(range(4), dist_channels):
    cross_sec[ch] = cross_array[:, i]
    u_cross_stat[ch] = u_cross_array_stat[:, i]
    u_cross_all[ch] = u_cross_array_all[:, i]

In [0]:
### which errors?
# fit_sigma = u_cross_stat
fit_sigma = u_cross_all

### Include errors of points in parameter errors?
absolute = True  # in general smaller errors
# absolute = False  # in general larger errors

def Breit_Wigner(E, M_Z, Gamma_Z, Peak, Underground):
    return Peak * (Gamma_Z*E)**2 / ((E**2 - M_Z**2)**2 + (E**2*Gamma_Z/M_Z)**2) + Underground

# dictionaries for fit parameters and their covariances
coeffs = {}
covs = {}
u_coeffs = {}
chisq = {}

# energy values for plotting the fits
E_axis = 2*np.linspace(44, 47, 501)


fig, ax = plt.subplots(1, 4, figsize=(28, 5))

for ch, i in zip(dist_channels, range(4)):
    # plot the cross sections
    ax[i].errorbar(x=2*E_lep, y=cross_sec[ch], yerr=fit_sigma[ch],
                   fmt='.', c='black', zorder=10,
                   label='measurements')

    # perform and plot fit
    coeffs[ch], covs[ch] = curve_fit(Breit_Wigner, 2*E_lep, cross_sec[ch],
                                     p0=[90, 2.5, 3, 0], maxfev=10000,
                                     sigma=fit_sigma[ch], absolute_sigma=absolute)
    chisq[ch] = np.sum((cross_sec[ch]-Breit_Wigner(2*E_lep, *coeffs[ch]))**2 / fit_sigma[ch]**2)
    u_coeffs[ch] = np.sqrt(np.diagonal(covs[ch]))

    ax[i].plot(E_axis, Breit_Wigner(E_axis, *coeffs[ch]), c=color[ch],
               label=r'Breit Wigner fit: $\chi^2/\mathrm{dof}=$'+'{0:4.2f}'.format(chisq[ch]/4))

    ax[i].set_title(plot_label[ch] + ' channel')
    ax[i].set_xlabel('energy $\sqrt{s}$ [GeV]')
    ax[i].set_ylabel('cross section $\sigma$ [nb]')

    ax[i].legend()

plt.show()


print('             M_Z             Gamma_Z             Peak             Offset')
for ch in dist_channels:
    print('{}: {:8.3f} +- {:4.3f}, {:7.3f} +- {:4.3f}, {:8.3f} +- {:4.3f}, {:8.3f} +- {:4.3f}'.format(
            ch,
            coeffs[ch][0], u_coeffs[ch][0],
            coeffs[ch][1], u_coeffs[ch][1],
            coeffs[ch][2], u_coeffs[ch][2],
            coeffs[ch][3], u_coeffs[ch][3])
         )

Now, determine compatibility of the fit parameters, defined for two values $A, B$ by $t = \frac{|A-B|}{\sqrt{\delta A^2 + \delta B^2}}$. This is the relative error of the difference $A-B$. Thus, we consinder the values compatible on a CL of 95% (resp. 99%) if $t\leq2$ (resp. $t\leq3$).

In [0]:
t_M_Z = abs(np.array(list(coeffs.values()))[:, 0][:, None]
            - np.array(list(coeffs.values()))[:, 0][None, :]) / (
    np.sqrt(np.array(list(u_coeffs.values()))[:, 0][:, None]**2
            + np.array(list(u_coeffs.values()))[:, 0][None, :]**2))

t_Gamma_Z = abs(np.array(list(coeffs.values()))[:, 1][:, None]
                - np.array(list(coeffs.values()))[:, 1][None, :]) / (
    np.sqrt(np.array(list(u_coeffs.values()))[:, 1][:, None]**2
            + np.array(list(u_coeffs.values()))[:, 1][None, :]**2))

print("Resonance mass compatibility: \n",
      np.array_repr(np.tril(t_M_Z),
                    precision=2,
                    suppress_small=True),
      '\n')

print("Resonance width compatibility: \n",
      np.array_repr(np.tril(t_Gamma_Z),
                    precision=2,
                    suppress_small=True),
      '\n')

In [0]:
### use values of quarks
M_Z = coeffs['qq'][0]
u_M_Z = u_coeffs['qq'][0]

Gamma_Z = coeffs['qq'][1]
u_Gamma_Z = u_coeffs['qq'][1]

# literature values (2020)
M_Z_PDG = 91.1876
u_M_Z_PDG = 0.0021

Gamma_Z_PDG = 2.4952
u_Gamma_Z_PDG = 0.0023

# compatibility
t_M_Z = abs(M_Z-M_Z_PDG)/np.sqrt(u_M_Z**2 + u_M_Z_PDG**2)
t_Gamma_Z = abs(Gamma_Z-Gamma_Z_PDG)/np.sqrt(u_Gamma_Z**2 + u_Gamma_Z_PDG**2)


# print results
print('M_Z:     {:7.3f} +- {:6.3f}, rel. error {:9.6f}; PDG: {:7.3f} +- {:6.3f}, t = {:1.2f}'.format(
        M_Z, u_M_Z, u_M_Z/M_Z,
        M_Z_PDG, u_M_Z_PDG, t_M_Z))
print('Gamma_Z: {:7.3f} +- {:6.3f}, rel. error {:9.6f}; PDG: {:7.3f} +- {:6.3f}, t = {:1.2f}'.format(
        Gamma_Z, u_Gamma_Z, u_Gamma_Z/Gamma_Z,
        Gamma_Z_PDG, u_Gamma_Z_PDG, t_Gamma_Z))


### Claculate partial widths

# peak hight in 1/GeV^2
nb_to_inv_GeV_sqrd = 1e-19 / ((sc.c*sc.hbar)/sc.e)**2  # unit conversion
Peak = {}
u_Peak = {}
for ch in dist_channels:
    Peak[ch] = nb_to_inv_GeV_sqrd*coeffs[ch][2]
    u_Peak[ch] = nb_to_inv_GeV_sqrd*u_coeffs[ch][2]

# theoretical prediction (from instructions)
part_width_theory = {
    'ee' : 0.0838,
    'mm' : 0.0838,
    'tt' : 0.0838,
    'qq' : 2*0.299+3*0.378,
    'nu' : 0.1676}

# dictionary for values
part_width = {}
u_part_width = {}

# for electrons different formula (since input=output)
part_width['ee'] = M_Z * Gamma_Z * np.sqrt(Peak['ee']/(12*np.pi))
u_part_width['ee'] = part_width['ee'] * np.sqrt((u_M_Z/M_Z)**2
                                                + (u_Gamma_Z/Gamma_Z)**2
                                                + (0.5*u_Peak['ee']/Peak['ee'])**2)

# for other channels
for ch in dist_channels[1:]:
    part_width[ch] = M_Z**2 * Gamma_Z**2 * Peak[ch] / (12 * np.pi * part_width['ee'])
    u_part_width[ch] = part_width[ch] * np.sqrt((2*u_M_Z/M_Z)**2
                                                + (2*u_Gamma_Z/Gamma_Z)**2
                                                + (u_Peak[ch]/Peak[ch])**2
                                                + (u_part_width['ee']/part_width['ee'])**2)

# compatibility
t_part_width = {}
for ch in dist_channels:
    t_part_width[ch] = abs(part_width[ch] - part_width_theory[ch]) / u_part_width[ch]

# print results
print('\npartial widths [MeV]')
for ch in dist_channels:
    print('{:5s}: {:6.1f} +- {:5.1f}, rel. error {:5.4f}; theory: {:6.1f}, t={:1.2f}'.format(
            ch,
            part_width[ch]*1000,
            u_part_width[ch]*1000,
            u_part_width[ch]/part_width[ch],
            part_width_theory[ch]*1000,
            t_part_width[ch]))


### Determine Number of neutrino families

# invisible decay width
Gamma_inv = Gamma_Z - sum(list(part_width.values()))
u_Gamma_inv = np.sqrt(u_Gamma_Z**2 +
                      np.sum(np.array(list(u_part_width.values()))**2))

# neutrino number (using theoretical prediction for Gamma_nu)
neutrino_number = Gamma_inv/part_width_theory['nu']
u_neutrino_number = neutrino_number * u_Gamma_inv/Gamma_inv

# print results
print('\n')
print('Gamma invisible [MeV]:      {:6.1f} +- {:5.1f}, rel. error {:3.2f}'.format(
        Gamma_inv*1000,
        u_Gamma_inv*1000,
        u_Gamma_inv/Gamma_inv))

print('Number of neutrino families:   {:2.1f} +- {:2.1f}, rel. error {:3.2f}'.format(
        neutrino_number,
        u_neutrino_number,
        u_neutrino_number/neutrino_number))

# Exercise 4: Forward-backward asymmetry and $\sin^2(\theta_\text{W})$ in muon final states

* Using the **muon channel only**, measure the forward-backward asymmetry $\mathcal{A}_\text{FB}$ using OPAL data and muon Monte Carlo events. Take into account the radiation corrections given below. 

| $\sqrt{s}$   \[GeV\]| Radiation correction [-]|  
| --- | --- | 
| 88.47 | 0.021512  | 
| 89.46 | 0.019262  | 
| 90.22 | 0.016713  | 
| 91.22 | 0.018293  | 
| 91.97 | 0.030286  | 
| 92.96 | 0.062196  | 
| 93.76 | 0.093850  | 

Feel free to use the dictionary 'radiation_corrections' given below.

* Measure the **Weinberg angle** as $\sin^2(\theta_\text{W})$. Compare the measurement with the literature.

In [0]:
radiation_corrections = { 'energy' : [ 88.47, 89.46, 90.22, 91.22, 91.97, 92.96, 93.76] ,
                          'correction' : [0.021512, 0.019262, 0.016713, 0.018293, 0.030286, 0.062196, 0.093850]}

In [0]:
# In the following consider only MUON-events:
# First, plot the cos_theta distribution for the mc- data and the opal data for Elep=45.62 GeV
array_E_lep_mm = {}

# select in opal data for muon events
for i in range(7):
    array_E_lep_mm[i] = {}  # sub dictionary for different variables
    # create muon mask
    mask = identify(array_E_lep[i],'mm')

    # mask arrays of all variables
    for variable in variable_names:
        array_E_lep_mm[i][variable] = array_E_lep[i][variable][mask]

# define variable and channel
variable = 'cos_thet'
channel = 'mm'

# bins for the histogram
bins_mc[variable] = np.linspace(-1,1, 201)

### Plotting
fig, ax = plt.subplots(1,2, figsize=(14, 5))

# Plot monte carlo data
ax[0].hist(array_mm[variable], bins=bins_mc[variable], histtype='step',
           linewidth=2, color=color[channel], label=plot_label[channel])

# Plot opal data
for j in [3]:
    ax[1].hist(array_E_lep_mm[j][variable], bins=bins_mc[variable], histtype='step',
               linewidth=2, label=r'$\mu$, ' +str(E_lep[j].round(2)) + ' GeV')


# Plot settings
ax[0].set_title(r'Cos$\theta$ - distribution, Monte Carlo-data')
ax[1].set_title(r'Cos$\theta$ - distribution, Opal-data')
ax[0].legend()
ax[1].legend()
plt.show()

In [0]:
# Function to determine weinberg-angle out of the forward backward asymmetry
# Simplification only valid for Elep \approx 45.6 GeV
def sin2_w(A):
    return (1-np.sqrt(A/3)) * 1/4

# Data managment for forward backward asymmetry
A_fb = {}
W_angle = {}

# Define maximal angle where the data will be cut
# Reference: "Zedometry", p.26
max_angle = 0.95

# Beam energies are near the resonance
print(r'E_lep_MC   =', array['mm']['E_lep'][0].round(3), 'GeV')
print(r'E_lep_OPAL =', E_lep[3].round(3), 'GeV\n')

### First, analyze MC data

# forward, backward and total event numbers
A_f = np.sum((array_mm['cos_thet']>=0) * (array_mm['cos_thet']<=max_angle))
A_b = np.sum((array_mm['cos_thet']>=-max_angle) * (array_mm['cos_thet']<=0))
N_fb = (A_f+A_b)

# Calculate forward-backward asymmetry
a_fb = (A_f-A_b) / N_fb
u_a_fb = np.sqrt((np.sqrt(A_f)/N_fb)**2
                 + (np.sqrt(A_b)/N_fb)**2
                 + ((A_f-A_b)*np.sqrt(N_fb)/N_fb**2)**2)

# Calculate Weinberg angle and uncertainty
w_angle = sin2_w(a_fb)
u_w_angle = 1/(8*np.sqrt(3)) * 1/np.sqrt(a_fb) * u_a_fb

# Fill the calculated values into dictionary
A_fb['mc'] = (a_fb, u_a_fb)
W_angle['mc'] = (w_angle, u_w_angle)


### Analogously for OPAL data

# Create subdictionaries for opal data at different Elep
A_fb['opal'] = {}
W_angle['opal'] = {}

# Iterate through all Elep energies
for i in range(7):

    # forward, backward and total event number
    A_f = np.sum((array_E_lep_mm[i]['cos_thet']>0) * (array_E_lep_mm[i]['cos_thet']<max_angle))
    A_b = np.sum((array_E_lep_mm[i]['cos_thet']>-max_angle) * (array_E_lep_mm[i]['cos_thet']<0))
    N_fb = (A_f+A_b)

    # Asymmetry
    a_fb = (A_f-A_b) / N_fb
    u_a_fb = np.sqrt((np.sqrt(A_f)/N_fb)**2
                     + (np.sqrt(A_b)/N_fb)**2
                     + ((A_f-A_b)*np.sqrt(N_fb)/N_fb**2)**2)
    # Correct for radiation
    a_fb = a_fb + radiation_corrections['correction'][i]

    # sin^2(Weinberg angle)
    w_angle = sin2_w(a_fb)
    u_w_angle = 1/(8*np.sqrt(3)) * 1/np.sqrt(a_fb) * u_a_fb

    # Save results in dictionary
    A_fb['opal'][i] = (a_fb, u_a_fb)
    W_angle['opal'][i] = (w_angle, u_w_angle)

# literature values
W_angle_PDG = 0.23122
u_W_angle_PDG = 0.00004

#compatibility between our OPAL and literature value
t_W_angle = abs(W_angle_PDG - W_angle['opal'][3][0]) / (
        np.sqrt(u_W_angle_PDG**2 + W_angle['opal'][3][1]**2))

print('            A_fb      , rel. err.;        sin^2(theta_W) , rel. error')
print('MC:   {:5.4f} +- {:5.4f}, {:5.4f}; {:15.4f} +- {:5.4f}, {:5.4f}'.format(
        A_fb['mc'][0],
        A_fb['mc'][1],
        A_fb['mc'][1]/A_fb['mc'][0],
        W_angle['mc'][0],
        W_angle['mc'][1],
        W_angle['mc'][1]/W_angle['mc'][0]))

print('OPAL: {:5.4f} +- {:5.4f}, {:5.4f}; {:15.4f} +- {:5.4f}, {:5.4f}'.format(
        A_fb['opal'][3][0],
        A_fb['opal'][3][1],
        A_fb['opal'][3][1]/A_fb['opal'][3][0],
        W_angle['opal'][3][0],
        W_angle['opal'][3][1],
        W_angle['opal'][3][1]/W_angle['opal'][3][0]))

print('PDG:                          ; {:15.4f} +- {:5.4f}, {:5.4f}; compatibility to OPAL: t = {:3.2f}'.format(
        W_angle_PDG,
        u_W_angle_PDG,
        u_W_angle_PDG/W_angle_PDG, t_W_angle))

In [0]:
# Plotting the forward-backward asymmetry
fig, ax = plt.subplots()

# plot monte carlo data point
ax.plot(array['mm']['E_lep'][0], A_fb['mc'][0], marker = '.', color ='C2', label='mc data')

# plot opal data
for i in range(7):
    if i==1:
        ax.errorbar(E_lep[i], A_fb['opal'][i][0], yerr=A_fb['opal'][i][1],
                    capsize=1, fmt='.', color ='C1', label='opal data')
    else:
        ax.errorbar(E_lep[i], A_fb['opal'][i][0], yerr=A_fb['opal'][i][1],
                    capsize=1, fmt='.', color ='C1')


# Plot settings
ax.set_xlabel(r'E_lep [GeV]')
ax.set_ylabel(r'$A_{FB}$')
ax.set_title(r'Forward-backward asymmetry')
ax.legend()

plt.show()

# Exercise 5: Tests on lepton universality¶

* Test the lepton universality from the total cross sectinos on the peak for $Z\to e^+ e^-$, $Z\to \mu^+ \mu^-$ and $Z\to \tau^+ \tau^-$ events. What is the ratio of the total cross section of the hadronic channel to the leptonic channels on the peak? Compare with the ratios obtained from the branching rations and discuss possible differences.

In [0]:
### analyse leptonic peak hights

for ch in dist_channels[:]:
    print('Peak {:}: {:6.4f} +- {:6.4f}, rel. error: {:5.4f}'.format(
            ch, coeffs[ch][2],
            u_coeffs[ch][2],
            u_coeffs[ch][2]/coeffs[ch][2]))

# compatibility matrix of peak hights
t_Peak = abs(np.array(list(coeffs.values()))[:3, 2][:, None]
             - np.array(list(coeffs.values()))[:3, 2][None, :]) / (
     np.sqrt(np.array(list(u_coeffs.values()))[:3, 2][:, None]**2
             + np.array(list(u_coeffs.values()))[:3, 2][None, :]**2))


print("\nLeptons peak compatibility: \n",
      np.array_repr(np.tril(t_Peak),
                    precision=2,
                    suppress_small=True),
      '\n')

only electron and tau compatible within one sigma, tau and muon compatible within 3 sigma, muon and electron within 4 sigma. Within our uncertainties lepton universality is not fulfuilled, BUT we did not yet consider systematic uncertainties (except for luminosity). ALso fits are not too good for anything else then tau. This deviation is explainable.

In [0]:
### comparison to branching ratios from literature (PDG)

# ratios Gamma_qq/Gamma_xx from PDG
R_PDG = {'ee' : 20.804,
         'mm' : 20.785,
         'tt' : 20.764}

u_R_PDG = {'ee' : 0.050,
           'mm' : 0.033,
           'tt' : 0.045}

# ratios of peak hights
R = {}
u_R = {}

#compatibility
t_R = {}

for ch in dist_channels[:3]:
    R[ch] = coeffs['qq'][2]/coeffs[ch][2]
    u_R[ch] = R[ch] * np.sqrt((u_coeffs[ch][2]/coeffs[ch][2])**2
                              + (u_coeffs['qq'][2]/coeffs['qq'][2])**2)

    t_R[ch] = abs(R_PDG[ch]-R[ch]) / np.sqrt(u_R[ch]**2 + u_R_PDG[ch]**2)

print('           our OPAL                            PDG                      compatibility')
for ch in dist_channels[:3]:
    print('qq/{:2s}: {:4.2f} +- {:3.2f}, rel. error {:5.3f}; {:5.3f} +- {:4.3f}, rel. error {:5.4f}, t={:1.2f}'.format(
            ch,
            R[ch],
            u_R[ch],
            u_R[ch]/R[ch],
            R_PDG[ch],
            u_R_PDG[ch],
            u_R_PDG[ch]/R_PDG[ch],
            t_R[ch]))